In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from iexfinance.refdata import get_symbols, get_iex_symbols

import os
import sys
import pandas as pd
import altair as alt
import datetime as dt

from pandas import to_datetime
from datetime import datetime, date
from iexfinance.stocks import get_historical_data
from dateutil.relativedelta import relativedelta

sys.path.append('..')
os.environ["START_DATE"] = "2010-08-25"
os.environ["END_DATE"] = "2020-08-25"

from analysis.src.per_high_low import get_min_max_dfs, compute_near_max_min, save_daily_results

In [3]:
# Used to log the `MESSAGES USED` in `_IEXBAse`.
# https://addisonlynch.github.io/iexfinance/stable/logging.html
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")
os.environ["IEX_LOG_LEVEL"] = "DEBUG"

DEBUG:root:test


In [18]:
IEX_TOKEN = "Tpk_57fa15c2c86b4dadbb31e0c1ad1db895"
os.environ["IEX_API_VERSION"] = "iexcloud-sandbox"
store = pd.HDFStore('dev_iex_store.h5')

In [5]:
all_symbols = store.keys()

In [6]:
print(store['AAPL'].index.min())
print(store['AAPL'].index.max())
print(store['NTZ'].index.min())

2006-12-29 00:00:00
2020-12-24 00:00:00
2006-12-29 00:00:00


In [7]:
import random
RANDOM_SAMPLE_SIZE = 3
symbols = random.sample(all_symbols, RANDOM_SAMPLE_SIZE)
symbols = ['AAPL'] + [symbol[1:] for symbol in symbols]

In [8]:
dfs_res = compute_near_max_min(symbols)
dfs_res

AAPL: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
AAPL: Data is catched between 2006-12-26 00:00:00 and 2020-12-29 00:00:00.
Successfully processed AAPL
BLV: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
BLV: Data is catched between 2006-12-27 00:00:00 and 2020-12-29 00:00:00.
Successfully processed BLV
PXSAP: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
PXSAP: Data is catched between 2006-12-28 00:00:00 and 2020-12-29 00:00:00.
Successfully processed PXSAP
WFC-P: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
WFC-P: Data is catched between 2006-12-30 00:00:00 and 2020-12-27 00:00:00.


/Users/olshansky/.pyenv/versions/3.7.7/envs/market_navigator_env/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'WFC-P'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


Successfully processed WFC-P
DONE computing min & max for 0.1
DONE computing min & max for 0.2


{0.1:             near_min  near_max
 date                          
 2013-03-19       NaN       NaN
 2013-03-20       NaN       NaN
 2013-03-21       NaN       NaN
 2013-03-22       NaN       NaN
 2013-03-25       NaN       NaN
 ...              ...       ...
 2020-12-15       0.0       1.0
 2020-12-21       0.0       1.0
 2020-12-22       0.0       1.0
 2020-12-23       0.0       1.0
 2020-12-24       0.0       1.0
 
 [1958 rows x 2 columns],
 0.2:             near_min  near_max
 date                          
 2013-03-19       NaN       NaN
 2013-03-20       NaN       NaN
 2013-03-21       NaN       NaN
 2013-03-22       NaN       NaN
 2013-03-25       NaN       NaN
 ...              ...       ...
 2020-12-15       0.0       1.0
 2020-12-21       0.0       1.0
 2020-12-22       0.0       1.0
 2020-12-23       0.0       1.0
 2020-12-24       0.0       1.0
 
 [1958 rows x 2 columns]}

In [9]:
# https://github.com/altair-viz/altair/issues/968
df_res = dfs_res[0.1]
df_alt = df_res.dropna(thresh=1)
df_alt = df_alt[df_alt.index>'2012-01-01']
data_alt = df_alt.reset_index().melt('date')

In [10]:
data_alt[data_alt['variable'] == 'near_min'].value.mean()

0.5360281195079086

In [11]:
# https://altair-viz.github.io/gallery/multiline_tooltip.html

import altair as alt
import pandas as pd

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')

# The basic line
line = alt.Chart(data_alt).mark_line(interpolate='basis').encode(
    x='date:T',
    y='value:Q',
#     color=alt.Color('color:N', scale = None)
    color='variable:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(data_alt).mark_point().encode(
    x='date:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'value:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(data_alt).mark_rule(color='gray').encode(
    x='date:T',
).transform_filter(
    nearest
)

# Horizontal average lines
avg_min = data_alt[data_alt['variable'] == 'near_min'].value.mean()
avg_max = data_alt[data_alt['variable'] == 'near_max'].value.mean()

line_avg_min = alt.Chart(pd.DataFrame({'value': [avg_min]})).mark_rule(strokeDash=[5,5]).encode(y='value')
line_avg_max = alt.Chart(pd.DataFrame({'value': [avg_max]})).mark_rule(strokeDash=[5,5]).encode(y='value')

# Put the five layers into a chart and bind the data
chart = alt.layer(
    line, selectors, points, rules, text, line_avg_min, line_avg_max
).properties(
    width=600, height=300
)


with alt.data_transformers.enable(max_rows=None):
    chart.display()
    #.save('chart.html', embed_options={'renderer':'svg'})

alt.LayerChart(...)

In [12]:
dfs_res = compute_near_max_min(symbols)

AAPL: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
AAPL: Data is catched between 2006-12-26 00:00:00 and 2020-12-29 00:00:00.
Successfully processed AAPL
BLV: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
BLV: Data is catched between 2006-12-27 00:00:00 and 2020-12-29 00:00:00.
Successfully processed BLV
PXSAP: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
PXSAP: Data is catched between 2006-12-28 00:00:00 and 2020-12-29 00:00:00.
Successfully processed PXSAP
WFC-P: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
WFC-P: Data is catched between 2006-12-30 00:00:00 and 2020-12-27 00:00:00.


/Users/olshansky/.pyenv/versions/3.7.7/envs/market_navigator_env/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'WFC-P'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


Successfully processed WFC-P
DONE computing min & max for 0.1
DONE computing min & max for 0.2
